This notebook takes the 1h price cleansed data from and enriches it with other context  

In [11]:
import pyspark.sql.functions as sf
from delta.tables import DeltaTable

In [12]:
# Read data from 'runescape.02_silver.1h_prices_cleansed'
df_1h_prices_cleansed = spark.read.table("runescape.02_silver.1h_prices_cleansed")

In [13]:
# Load item mapping data from the Silver Layer
df_item_mapping = spark.read.table("runescape.02_silver.item_mapping")

In [14]:
# enrich price data with map data
df_1h_prices_enriched_updates = df_1h_prices_cleansed.join(df_item_mapping, "id")

Should the data be limited to the last time only?  
This could improve performance?

In [15]:
# write latest data to table

targetDF = DeltaTable.forName(spark, "runescape.02_silver.1h_prices_enriched")
dfUpdates = df_1h_prices_enriched_updates

(targetDF.alias("t")
  .merge(
    source = dfUpdates.alias("s"),
    condition = "t.id = s.id AND t.time = s.time")
  .whenNotMatchedInsertAll()
  .execute())

,num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
0,0,0,0,0
